In [111]:
import json

In [112]:
load = lambda x: json.load(open(x))

In [161]:
all_results = !ls results/llm_eval_openllm/ | grep report | grep optimized-

In [162]:
reports = [ load(f'results/llm_eval_openllm/{result_file}') for result_file in all_results]

In [163]:
len(reports)

4

In [164]:
settings = [ x.split('-') for x in all_results]

In [165]:
settings = [ [ y.split('.')[0] for y in x] for x in settings]

In [166]:
settings = [ dict(zip(['model', 'prompt', 'split', 'story'], x)) for x in settings]

In [167]:
import pandas as pd

In [168]:
get_reg = lambda x:{ p.replace('_human_AGG','').replace('similarity_',''):q for p,q in { k:v['mean']['regression'] for k,v in x.items()}.items()}

In [169]:
get_cls = lambda x:{ p.replace('_human_AGG','').replace('similarity_',''):q for p,q in {k: v['mean']['classification'] for k,v in x.items()}.items()}

In [170]:
# flatten(get_cls(report['similarity_empathy_human_AGG']))

In [171]:
def merge(dict1, dict2):
    res = {**dict1, **dict2}
    return res

In [172]:
def merge_more(*dicts):
    d = {}
    for x in dicts:
        d = merge(d,x)
    return d

In [173]:
def merge_values(dict1, dict2):
    merged_dict = {}
    for k in dict1.keys():
        merged_dict[k] = merge_more(dict1[k], dict2[k])
    return merged_dict

In [177]:
print(pd.DataFrame(merge_values(get_reg(reports[2]), get_cls(reports[2]))).T.to_latex(float_format=lambda x:f'{x:.3f}'))

\begin{tabular}{lrrrrrrr}
\toprule
 & Pearson & Spearman & MSE & accuracy & precision & recall & F1 \\
\midrule
empathy & 0.304 & 0.295 & 0.970 & 0.545 & 0.628 & 0.534 & 0.436 \\
event & 0.337 & 0.357 & 1.980 & 0.305 & 0.625 & 0.547 & 0.287 \\
emotion & 0.305 & 0.312 & 1.180 & 0.495 & 0.617 & 0.532 & 0.407 \\
moral & 0.305 & 0.320 & 1.320 & 0.455 & 0.659 & 0.545 & 0.391 \\
\bottomrule
\end{tabular}



In [175]:
settings

[{'model': 'meta_llama_Meta_Llama_3_70B_Instruct',
  'prompt': 'optimized',
  'split': 'dev',
  'story': 'full'},
 {'model': 'meta_llama_Meta_Llama_3_70B_Instruct',
  'prompt': 'optimized',
  'split': 'dev',
  'story': 'summary'},
 {'model': 'meta_llama_Meta_Llama_3_70B_Instruct',
  'prompt': 'optimized',
  'split': 'test',
  'story': 'full'},
 {'model': 'meta_llama_Meta_Llama_3_70B_Instruct',
  'prompt': 'optimized',
  'split': 'test',
  'story': 'summary'}]

In [44]:
flatten = lambda x: { f'{p}-{k}':q for k,v in x.items()  for p,q in v.items()}

In [34]:
for i,(report, setting) in enumerate(zip(reports, settings)):
    # setting['reg'] = pd.DataFrame(get_reg(report['similarity_empathy_human_AGG']))
    setting = merge_more(setting, flatten(get_reg(report)), flatten(get_cls(report)))
    settings[i] = setting

KeyError: 'mean'

In [26]:
df = pd.DataFrame(settings).sort_values(['split', 'prompt', 'story'])

In [27]:
df

,model,prompt,split,story,Pearson-mean,Spearman-mean,MSE-mean,Pearson-standard,Spearman-standard,MSE-standard,...,recall-mean,F1-mean,accuracy-standard,precision-standard,recall-standard,F1-standard,accuracy-reverse,precision-reverse,recall-reverse,F1-reverse
0,meta_llama_Meta_Llama_3_70B_Instruct,optimized,dev,full,0.264,0.284,1.11,0.254,0.251,1.14,...,0.554,0.421,0.570,0.673,0.638,0.563,0.500,0.586,0.566,0.490
1,meta_llama_Meta_Llama_3_70B_Instruct,optimized,dev,summary,0.450,0.459,0.61,0.401,0.418,0.65,...,0.717,0.697,0.740,0.725,0.729,0.727,0.720,0.712,0.723,0.713
2,meta_llama_Meta_Llama_3_70B_Instruct,optimized,test,full,0.304,0.295,0.97,0.300,0.289,0.98,...,0.534,0.436,0.580,0.659,0.571,0.507,0.557,0.610,0.548,0.479
3,meta_llama_Meta_Llama_3_70B_Instruct,optimized,test,summary,0.405,0.403,0.62,0.397,0.387,0.66,...,0.630,0.614,0.652,0.654,0.651,0.650,0.623,0.624,0.620,0.619


In [19]:
setting_column = ['model', 'split', 'prompt', 'story']

In [20]:
column_blacklist = ['MSE', 'precision', 'recall']

In [21]:
ordered_columns = list(filter(lambda u: all([ r not in u for r in column_blacklist]), setting_column + sorted([x for x in df.columns if x not in setting_column])))

In [22]:
df = pd.DataFrame(settings).sort_values(['split', 'prompt', 'story'])[ordered_columns]

In [23]:
df['model'] = 'Llama3-70B'

In [25]:
df.to_csv('results/llama3_70b.csv',float_format=lambda x:f'{x:.2f}')

In [ ]:
load('results/llm_eval_openllm/meta_llama_Meta_Llama_3_70B_Instruct-optimized-dev-summary.report.json')